In [ ]:
import sqlite3 as lite
import requests
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import pandas as pd
import time
import datetime
from dateutil.parser import parse 
import collections

r = requests.get('http://www.citibikenyc.com/stations/json')
df = json_normalize(r.json()['stationBeanList'])
con = lite.connect('citi_bike.db')
cur = con.cursor()

def repeat_query():

    r = requests.get('http://www.citibikenyc.com/stations/json')
    exec_time = parse(r.json()['executionTime']).strftime("'%Y-%m-%d %H:%M:%S'")
    
    for i in range(60):

        cur.execute('INSERT INTO available_bikes (execution_time) VALUES (?)', (exec_time,))

        for station in r.json()['stationBeanList']:
            cur.execute("UPDATE available_bikes SET _%d = %d WHERE execution_time = %s" % (station['id'], station['availableBikes'], exec_time))
        con.commit()

        time.sleep(60)

    con.close()

#----------------------------------------------------------------------------------------------------------------------------
    
with con:
    cur.execute('''DROP TABLE citibike_reference''')
    cur.execute('''DROP TABLE available_bikes''')
    cur.execute('''CREATE TABLE citibike_reference (id INT PRIMARY KEY, totalDocks INT, city TEXT, altitude INT, stAddress2 TEXT, 
                longitude NUMERIC, postalCode TEXT, testStation TEXT, stAddress1 TEXT, stationName TEXT, landMark TEXT,
                latitude NUMERIC, location TEXT )''')

#a prepared SQL statement we're going to execute over and over again
sql = '''INSERT INTO citibike_reference (id, totalDocks, city, altitude, stAddress2, longitude, 
postalCode, testStation, stAddress1, stationName, landMark, latitude, location) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)'''

#for loop to populate values in the database
with con:
    for station in r.json()['stationBeanList']:
        #id, totalDocks, city, altitude, stAddress2, longitude, postalCode, testStation, stAddress1, stationName, landMark, latitude, location)
        cur.execute(sql,(station['id'],station['totalDocks'],station['city'],
        station['altitude'],station['stAddress2'],station['longitude'],station['postalCode'],
        station['testStation'],station['stAddress1'],station['stationName'],station['landMark'],station['latitude'],
        station['location']))
        
#extract the column from the DataFrame and put them into a list
station_ids = df['id'].tolist() 

#add the '_' to the station name and also add the data type for SQLite
station_ids = ['_' + str(x) + ' INT' for x in station_ids]

#create the table - in this case, we're concatenating the string and joining all the station ids (now with '_' and 'INT' added)
with con:
    cur.execute("CREATE TABLE available_bikes ( execution_time INT, " +  ", ".join(station_ids) + ");")
    
#take the string and parse it into a Python datetime object
exec_time = parse(r.json()['executionTime'])

with con:
    cur.execute('INSERT INTO available_bikes (execution_time) VALUES (?)', (exec_time.strftime('%Y-%m-%d %H:%M:%S'),))
    
id_bikes = collections.defaultdict(int) #defaultdict to store available bikes by station

#loop through the stations in the station list
for station in r.json()['stationBeanList']:
    id_bikes[station['id']] = station['availableBikes']

# iterate through the defaultdict to update the values in the database
with con:
    for k, v in id_bikes.iteritems():
        cur.execute("UPDATE available_bikes SET _" + str(k) + " = " + str(v) + " WHERE execution_Time = " + str((exec_time-datetime.datetime(1970,1,1)).total_seconds(),) + ";")
        
repeat_query()